In [84]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer 

First off all, we need to read the data using pandas function ".read_csv".
We need to use the read.csv function since the file's extension we'll be using is csv 

In [85]:
df = pd.read_csv("DataSets/GPA.csv")

Secondly, we need to know the shape and the type of data. So, we can get into exploring and analysing it.

In [86]:
print(df.shape)
print(df.info())

(193, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Gender       189 non-null    object 
 1   study_hours  189 non-null    float64
 2   gpa          189 non-null    float64
dtypes: float64(2), object(1)
memory usage: 4.7+ KB
None


As seen, this dataset contrains 3 columns followed alongside with 193 rows (entries). 2 of columns (gpa & study_hours) contain numerical values, while 1 of them (gender) contains categorical values. However, 4 of the rows of each column are null values. We can also check for null values that way:

In [87]:
print(df.isnull().sum())

Gender         4
study_hours    4
gpa            4
dtype: int64


As expected, we have 12 null values in total. Thus, we need to check for duplicated values before minding the null values.

In [88]:
print(df.duplicated().sum())

26


We have 26 duplicated values which we need to get rid of them before doing anything complicated. We can check for duplicated values seperately by each column alone. However, it is more recommended to check for all once.

In [89]:
df_clean = df.drop_duplicates()
print(df_clean.shape)

(167, 3)


Success! As we know, there were 26 duplicated values. Therefore, after dropping them the total entries we have now is 167. If you add them both together, the result will be 193, which is the total of the entries with the duplicated values includer.

Back to null values. Altough, after dropping the duplicates. We need to check for the null values again since some of them could've been duplicateds. (which were dropped)

In [90]:
print(df_clean.isnull().sum())

Gender         4
study_hours    4
gpa            4
dtype: int64


Alright, so none of them were dropped. Most likely, in categorical null values cases, they are solved by filling the most frequent value with the null value. If we want to check for the most frequent categorical value, we can use that method. Though, we will only be checking for the gender column's entries since it's the only column that contains categorical values.

In [91]:
print(df_clean.Gender.value_counts())

Gender
Male      125
Female     38
Name: count, dtype: int64


As seen, the most frequent categorical value is the gener "Male".

We can replace each categorical null value in the column ("Gender") with the most frequent value ("Male") using simple imputer from the sklearn module.

In [92]:
from sklearn.impute import SimpleImputer 

# In gender, there is 4 null values. Solution:

from sklearn.impute import SimpleImputer # specialized in handling missing values

categorical_imputer = SimpleImputer(strategy="most_frequent")

df_clean[["Gender"]] = categorical_imputer.fit_transform(df_nodups[["Gender"]])

C:\Users\htami\AppData\Local\Temp\ipykernel_16272\3401033860.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[["Gender"]] = categorical_imputer.fit_transform(df_nodups[["Gender"]])


Alright, should be changed. Let's check!

In [93]:
df_clean.isnull().sum()

Gender         0
study_hours    4
gpa            4
dtype: int64

Yup, as expected. All null values have been replaced with the most frequent one. ("Male")

Now, we need to focus on the numerical values. (gpa and study_hours column). In numerical values' cases, the null values are most likely replaced by the mean ("average") of the data in the column. However, in some cases, that is completely unfair. The second solution is by taking other information from adjacent columns. As we know, the dataset has three columns so each column has 2 sources to grab extra information from. Let's start by comparing the mean of the gpa column with the mean of the gpa column but filtered with the gender and the study_hours.

In [94]:
print(df_clean["gpa"].mean())
precise_filter = df_clean[(df_clean["Gender"] == "Male") & (df_clean["study_hours"] >= 17)]
print(precise_filter["gpa"].mean())

3.5767423312883437
3.6174166666666667


We set the study hours for 3 for example (just to keep in mind the difference). As well as the gender for male. The differnce in the values are 0.4 only. In such cases, considering the mean is the best solution. Now, we need to do the same thing but with the study hours.

In [95]:
print(df_clean["study_hours"].mean())
precise_filter = df_clean[(df_clean["Gender"] == "Male") & (df_clean["gpa"] >= 3.2)]
print(precise_filter["study_hours"].mean())

17.588957055214724
17.896396396396398


Here, as well we set some filters with random values. The difference is almost 0.31, which is not a big deal at all. Thus, we will use the simple imputer with the "mean" strategy. As explained why above.

In [96]:
from sklearn.impute import SimpleImputer # specialized in handling missing values

categorical_imputer = SimpleImputer(strategy="mean")

df_clean[["gpa", "study_hours"]] = categorical_imputer.fit_transform(df_clean[["gpa", "study_hours"]])

C:\Users\htami\AppData\Local\Temp\ipykernel_16272\3008559515.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[["gpa", "study_hours"]] = categorical_imputer.fit_transform(df_clean[["gpa", "study_hours"]])


Let's take an eye real quick at the null values now.

In [97]:
print(df_clean.isnull().sum())

Gender         0
study_hours    0
gpa            0
dtype: int64


Yes! Zeros.

Now, our next step is to actually explore the data. Let's start by using the .descrbie function to get some mathematical interesting information.

In [98]:
print(df_clean.describe())

       study_hours         gpa
count   167.000000  167.000000
mean     17.588957    3.576742
std      11.396195    0.286676
min       2.000000    2.600000
25%      10.000000    3.400000
50%      15.000000    3.600000
75%      20.000000    3.800000
max      69.000000    4.300000


The describe function is only specialized in numerical values. That's why the gender column isn't there (it's a categorical column). Let's start by exploring the study_hours column, it basically indicates the study hours students spent studying. The total count of the study hours columns entries is 167. The average of the study hours per studern is about 18 hours which is realistic for a normal/ good student (averages about 2.5 hours per day), if we're considering that these are their study hours per week which is the most realistic option. The minmum hours a student has spent is 2 hours, and in return he got a 2.6 grade average (gpa). The highest amount of hours a student has studies is 69 hours, has an average of almost 10 hours per day. (which indicates that the student is extremely dedicated and determined) In return, he got 4.3 which is considered an outlier grade (will get to that topic later). There is also some interesting information such as what does 25, 50, 75% of the students study. As well, as the STR (short tandem repeats) for the students' study hours.

Secondly, the gpa (stands for "grade per point") column, which indicates the total grade of students. The process is similar to the study hours column. First of all, the count of the entire rows (entries) equals 167 (which makes sense, supposed to be smilar to the other entries in other columns). The average of the grades is 3.5 which is something impressive since the full mark is most likely 4. The maximum grade a student has ever got is 4.3 (an outlier value). The student whose mark is 4.3 has studied 69 hours, averaging almost 10 hours per day. There is also as well some interesting information such as what does 25, 50, 75% of the students study. As well, as the STR (short tandem repeats) for the students' gpa.

Exploring Outliers. Outliers are considered exceptions which are most likely inputting mistakes. Outliers are explored theoretically, and by common sense. For example, the 4.3 grade. In gpa ratious, there isn't any 4.3 grades. It must be an integer number. There is only 2 

In [99]:
filter_outliers = df[(df["gpa"] > 4) | (df["gpa"] < 0)]

print(filter_outliers)

    Gender  study_hours  gpa
79  Female         10.0  4.3


 Outliers are considered exceptions which are most likely inputting mistakes. Outliers are explored theoretically, and by common sense (must use filters). In this dataset, there is only one outlier value which is "4.3". Since in this dataset, full marks are 4. Now, it comes down to 2 options: The original mark is either 3.4 (highest chance). Or, that student, got extra marks. However, what's the chance of getting a 0.3 bonus, or getting a 1+ bonus and losing 0.7 a mark? In such cases, it's recommended to contact the original source of the dataset. Secondly, The outlier data needs to be replaced with the option that is valid and makes sense:

In [100]:
df_clean = df_clean["gpa"].replace(4.3, 3.4)

Done! This was the process of exploring the data, containing the most important and useful methods.